In [ ]:
import os
import requests
from dotenv import load_dotenv
import fitz
from bs4 import BeautifulSoup
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
from openai import OpenAI

load_dotenv()
upstage_api_key = os.getenv("UPSTAGE_API_KEY")

In [13]:
def split_pdf(input_file, batch_size):
    # Open input_pdf
    input_pdf = fitz.open(input_file)
    num_pages = len(input_pdf)
    print(f"Total number of pages: {num_pages}")
 
    # Split input_pdf
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1
 
        # Write output_pdf to file
        input_file_basename = os.path.splitext(input_file)[0]
        output_file = f"{input_file_basename}_{start_page}_{end_page}.pdf"
        print(output_file)
        with fitz.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
 
    # Close input_pdf
    input_pdf.close()

In [9]:
# Input arguments
input_file = "lloydk_Q&A.pdf" # Replace with a file of your own
batch_size = 1  # Maximum available value is 100
split_pdf(input_file, batch_size)

Total number of pages: 10
lloydk_Q&A_0_0.pdf
lloydk_Q&A_1_1.pdf
lloydk_Q&A_2_2.pdf
lloydk_Q&A_3_3.pdf
lloydk_Q&A_4_4.pdf
lloydk_Q&A_5_5.pdf
lloydk_Q&A_6_6.pdf
lloydk_Q&A_7_7.pdf
lloydk_Q&A_8_8.pdf
lloydk_Q&A_9_9.pdf


In [10]:
# 테이블을 구분하는 코드

tables = []
paragraphs = []

for i in range(10):
    filename = f"lloydk_Q&A_{i}_{i}.pdf"
    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {upstage_api_key}"}
    files = {"document": open(filename, "rb")}
    data = {"ocr": "force", "base64_encoding": "['table']", "model": "document-parse"}
    response = requests.post(url, headers=headers, files=files, data=data)
    for element in response.json()['elements']:
        if element['category'] == 'table':
            table_html = element['content']["html"]
            soup = BeautifulSoup(table_html, "html.parser")
            # rows = []
            for tr in soup.find_all("tr"):
                cells = [
                    td.get_text(strip=True).replace("\n", " ")
                    for td in tr.find_all(["td", "th"])
                ]
                paragraphs.append(cells)
            # for r in rows:
            #     paragraphs.append(r)
        else:
            html_str = element['content']['html']
            if html_str.startswith('"') and html_str.endswith('"'):
                html_str = html_str[1:-1]
            soup = BeautifulSoup(html_str, "html.parser")
            text = soup.get_text(" ", strip=False)
            paragraphs.append(text)

In [11]:
blocks = []
current_type = None   # 'text' 혹은 'table'
current_items = []

def item_type(item):
    # 문자열이면 문장
    if isinstance(item, str):
        return "text"
    # 리스트면 (여기서는) 테이블
    if isinstance(item, list):
        return "table"
    return "other"

for item in paragraphs:
    t = item_type(item)

    # text / table / other 타입이 바뀌면 이전 블록을 마무리
    if t != current_type:
        if current_items:
            blocks.append({
                "type": current_type,
                "items": current_items
            })
        current_type = t
        current_items = []

    current_items.append(item)

# 마지막 블록도 추가
if current_items:
    blocks.append({
        "type": current_type,
        "items": current_items
    })


In [12]:
text_for_embedding = []
for i in range(len(blocks)):
    texts = []
    if blocks[i]['type'] == 'text':
        texts.append(blocks[i]['items'])
        text_for_embedding.append(texts[0])
    else:
        texts = []
        texts.append(blocks[i-1]['items'][-1:]+blocks[i]['items'])
        text_for_embedding.append(texts[0])

In [31]:
def chunk_by_char_with_overlap(lines, max_chars=700, overlap_chars=50):
    """
    lines: 이미 문장 단위로 나뉜 리스트 (list[str])
    max_chars: 청크 하나의 최대 글자 수
    overlap_chars: 다음 청크로 넘길 최소 글자 수 (겹치는 분량)
    """
    chunks = []
    current = []
    cur_len = 0

    for raw in lines:
        line = raw.strip()
        if not line:
            continue

        line_len = len(line)

        # 이 줄을 더하면 max_chars를 넘는 경우 → 지금까지 걸 하나의 청크로 확정
        if current and (cur_len + line_len > max_chars):
            # 현재 청크 저장
            chunks.append("\n".join(current))

            # 🔁 오버랩 부분 만들기: 뒤에서부터 overlap_chars 이상이 될 때까지 가져오기
            overlap = []
            overlap_len = 0
            for s in reversed(current):
                if overlap_len + len(s) > overlap_chars and overlap:
                    break
                overlap.append(s)
                overlap_len += len(s)
            overlap = list(reversed(overlap))

            current = overlap[:]          # 새 청크는 오버랩으로 시작
            cur_len = sum(len(s) for s in current)

        # 현재 청크에 이 줄 추가
        current.append(line)
        cur_len += line_len

    # 마지막 청크 처리
    if current:
        chunks.append("\n".join(current))

    return chunks

In [41]:
list_for_chunks = []
for chunks in text_for_embedding:
    if type(chunks[1]) == list:
        list_for_chunks.append(chunks)
        continue
    else:
        num_chunks = chunk_by_char_with_overlap(chunks)
        for num in num_chunks:
            list_for_chunks.append(num)

In [7]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 .env에 없어요 ㅠㅠ")

client_oa = OpenAI(api_key=OPENAI_API_KEY)

# Qdrant 클라이언트 (로컬)
client_qd = QdrantClient(
    url="http://localhost:6333",  # 또는 host="localhost", port=6333
)

EMBED_MODEL = "text-embedding-3-small"
EMBED_DIM = 1536  # text-embedding-3-small의 차원 수
COLLECTION_NAME = "lloydk_docs"

In [60]:
def ensure_collection():
    print("🔍 Checking collection existence...")
    exists = client_qd.collection_exists(collection_name=COLLECTION_NAME)
    print(f"→ Exists? {exists}")

    if not exists:
        client_qd.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=EMBED_DIM, distance=Distance.COSINE),
        )
        print(f"✅ Created collection: {COLLECTION_NAME}")
    else:
        print(f"⚙️ Collection already exists: {COLLECTION_NAME}")
        
ensure_collection()

🔍 Checking collection existence...
→ Exists? True
⚙️ Collection already exists: lloydk_docs


In [5]:
def embed_batch(texts):
    """
    texts: list[str]
    return: list[list[float]] (임베딩 벡터들)
    """
    # 빈 문자열 제거 + 전처리
    cleaned = []
    idx_map = []

    for i, t in enumerate(texts):
        if not isinstance(t, str):
            t = str(t)
        t = t.strip()
        if not t:
            continue
        cleaned.append(t)
        idx_map.append(i)

    if not cleaned:
        return [], []

    resp = client_oa.embeddings.create(
        model=EMBED_MODEL,
        input=cleaned,
    )

    vectors = [d.embedding for d in resp.data]
    return vectors, cleaned

In [8]:
BATCH_SIZE = 100
for i in range(0, len(list_for_chunks), BATCH_SIZE):
    batch = list_for_chunks[i:i+BATCH_SIZE]
    vectors, cleaned = embed_batch(batch)

    points = [
        PointStruct(
            id=str(uuid.uuid4()),
            vector=vec,
            payload={"text": text},
        )
        for text, vec in zip(cleaned, vectors)
    ]

    client_qd.upsert(collection_name="lloydk_docs", points=points)

NameError: name 'list_for_chunks' is not defined

In [41]:
query = "컨텍스트 윈도우가 가장 긴 언어모델"
q_emb, _ = embed_batch([query])

results = client_qd.query_points(
    collection_name="lloydk_docs",
    query=q_emb[0],   # query_vector → query
    limit=3,
    with_payload=True
)

for r in results.points:
    print(f"score: {r.score:.3f}")
    print(f"text: {r.payload['text']}\n")


score: 0.584
text: · 컨텍스트 윈도우 (Context Window)
컨텍스트 윈도우는 한 번에 처리할 수 있는 입력 길이로, 대화나 문맥을 얼마나 길게 기억할 수 있는 지를 뜻합니다.
· LLM의 사양과 성능 간의 상관관계
일반적으로 파라미터 수가 많을수록 성능이 좋아지지만, 일정 수준 이상에서는 데이터 품질, 학습 방 법이 더 중요해집니다. 즉, 크기가 성능에 영향을 주긴 하지만, '무조건 클수록 좋은 건 아님'이 최근 트렌드입니다.
· 사양과 비용 간의 상관관계
파라미터 수가 많을수록 연산량과 메모리 사용량이 급증해 학습·추론 비용도 높아집니다. 따라서 고사양 모델은 높은 성능을 주지만, 실사용 시 운영비용과 응답 속도 문제도 함께 고려해야 합니다.
· 대표적인 LLM 모델들의 파라미터 수와 컨텍스트 윈도우 길이

score: 0.548
text: ['· 대표적인 LLM 모델들의 파라미터 수와 컨텍스트 윈도우 길이', ['모델명', '파라미터 수', '컨텍스트 윈 도우', '특징', '적합한 용도'], ['Llama 4 (Maverick)', '400B', '1M', '초대형 다국 어 대응, 멀티 모달 처리', '글로벌 대용 량 문서 분석, RAG'], ['DeepSeek R1', '671B (MoE)', '128K', '추론 특화, 비 용 효율', '고난도 질문 응답, 분석 도 구'], ['Claude 3.7 Sonnet', '(비공개)', '350K', '안전성과 판 단력 강화', '챗봇, 문서 정 리, 비서형 AI'], ['Mistral Small 3.1', '24B', '128K', '가볍고 빠름, 오픈소스', '내부 시스템 연동, 엣지 디 바이스'], ['Phi-4', '14.7B', '16K', '저사양 환경 대응', '임베디드 AI, 비용 민감 환 경'], ['GPT-4o', '1.8T (추정)', '2M', '텍스트+이미 지 동시 처리', '최고 성능, 복 합 AI 서비스'], ['XGen-7B', '7B', '8K', '중소

In [42]:
def rag_chat():
    print("RAG 챗봇입니다. 'exit' 입력 시 종료.\n")
    while True:
        q = input("👤 질문: ").strip()
        if not q:
            continue
        if q.lower() in ["exit", "quit", "q"]:
            print("bye~")
            break

        # 1️⃣ 쿼리 임베딩
        q_emb, _ = embed_batch([q])

        # 2️⃣ Qdrant에서 검색
        results = client_qd.query_points(
            collection_name="lloydk_docs",
            query=q_emb[0],
            limit=5,
            with_payload=True
        )

        # 3️⃣ 검색된 문서 모으기
        contexts = [r.payload["text"] for r in results.points if "text" in r.payload]
        if not contexts:
            print("🤖 관련 문서를 찾지 못했어요 ㅠㅠ\n")
            continue

        # 4️⃣ 답변 생성
        context_text = "\n\n".join(contexts)
        system_prompt = (
            "너는 DO 솔루션 관련 기술 문서를 바탕으로 답변하는 어시스턴트야.\n"
            "반드시 아래 제공된 문맥 내에서만 답변해. 모르면 모른다고 말해.\n"
            "table의 경우, [[열1, 열2], [열1에 대한 아이템, 열2에 대한 아이템]...] 이런 식으로 되어있으니 반드시 끝까지 다 보고 답변해야 해."
        )
        user_prompt = (
            f"[질문]\n{q}\n\n[관련 문서]\n{context_text}\n\n"
            "위 내용을 기반으로 한국어로 자연스럽게 답변해줘."
        )

        resp = client_oa.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
        )

        print("\n🤖 답변:\n", resp.choices[0].message.content, "\n")


In [43]:
rag_chat()

RAG 챗봇입니다. 'exit' 입력 시 종료.


🤖 답변:
 현재 컨텍스트 윈도우가 가장 긴 언어 모델은 "GPT-4o"로, 2M의 컨텍스트 윈도우를 가지고 있습니다. 이 모델은 텍스트와 이미지를 동시에 처리할 수 있는 복합 AI 서비스에 적합한 최고 성능의 모델로 알려져 있습니다. 

bye~
